In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
req=Request("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html=urlopen(req).read()

In [13]:
soup=BeautifulSoup(html,'html.parser')
dataset=soup.find('table',{'class':'wikitable sortable'})
data1=dataset.findAll("tr")[:1]
header_list=[]
post_code=[]
prev_post=""
prev_borough=""
prev_neighbour=""
for row in dataset.findAll("tr")[1:]:
	col=row.findAll("td")
	if col[1].text!='Not assigned' and 'Not assigned' in col[2].text:

		header_list=[col[0].text,col[1].text,col[1].text]
	else:
		if col[1].text!='Not assigned' and (prev_post==col[0].text) and (prev_borough==col[1].text):
				header_list=[col[0].text,col[1].text,(prev_neighbour+","+col[2].text)]
				prev_neighbour=prev_neighbour+','+col[2].text
		else:
			if col[1].text!='Not assigned':
				header_list=[col[0].text,col[1].text,col[2].text]
				prev_neighbour=col[2].text
			else:
				continue
			
	post_code.append(header_list)
	prev_post=col[0].text
	prev_borough=col[1].text
table_df=pd.DataFrame(post_code,columns=['PostalCode','Borough','Neighbourhood'])
table_df = table_df.replace(r'\n','', regex=True) 
table_df.drop_duplicates(['PostalCode'],keep='last',inplace=True)
print(table_df.shape)

(103, 3)


In [4]:
pip install geocoder

  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
  Stored in directory: C:\Users\Sandip-13760\AppData\Local\pip\Cache\wheels\0c\61\d2\d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [7]:
geo_coordinate_df=pd.read_csv('D:\Coursera\Geospatial_Coordinates.csv')  # Download the file

In [12]:
result_df=pd.merge(table_df,geo_coordinate_df,left_on='PostalCode',right_on='Postal Code',how='left')
result_df=result_df[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
print(result_df.head(3))

  PostalCode           Borough             Neighbourhood   Latitude  Longitude
0        M3A        North York                 Parkwoods  43.753259 -79.329656
1        M4A        North York          Victoria Village  43.725882 -79.315572
2        M5A  Downtown Toronto  Harbourfront,Regent Park  43.654260 -79.360636
